In [1]:
import os
import sys
import numpy as np


# Root directory of the project
ROOT_DIR = os.path.abspath("../../")
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from openimages import OpenImageDataset
from openimages import OpenImagesConfig
import openimages as oim
from tqdm import tqdm

/home/bartosz/.pyenv/versions/3.7.4/envs/open-images/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bartosz/.pyenv/versions/3.7.4/envs/open-images/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bartosz/.pyenv/versions/3.7.4/envs/open-images/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

In [2]:
class InferenceConfig(OpenImagesConfig):
    """
    """
    # Give the configuration a recognizable name
    NAME = "openimages"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
inference_config = InferenceConfig()

In [3]:
# Recreate the model in inference mode
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

import tensorflow as tf

with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", 
                              config=inference_config,
                              model_dir=MODEL_DIR)
  
# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()
#model_path = os.path.join(ROOT_DIR, 'logs/openimages20190926T0931/mask_rcnn_openimages_0018.h5')

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)






Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights from  /home/bartosz/open-images/Mask_RCNN/logs/openimages20190926T0931/mask_rcnn_openimages_0106.h5
Re-starting from epoch 106


In [4]:
PATH = os.path.abspath(os.path.join(ROOT_DIR, '../data/'))

# Validation dataset
dataset = OpenImageDataset()
dataset.load_dataset(PATH, 'kaggle-test')
dataset.prepare()

In [5]:
import datetime
now = datetime.datetime.utcnow()
datestr = now.strftime("%Y%m%dT%H%M")
submission_file = "submission-{}.csv".format(datestr)

header = "ImageID,ImageWidth,ImageHeight,PredictionString"
n_rows = 0
with open(submission_file, 'w') as fid:
    fid.write(header+'\n')

In [6]:
len(dataset.image_ids)

99999

submission_file = 'submission-20190927T1135.csv'
n_rows = 0
with open(submission_file, 'r') as fid:
    for line in fid:
        n_rows += 1
last_row = line
n_rows -= 1

In [8]:


with open(submission_file, 'a') as fid:
    
    for image_id in tqdm(dataset.image_ids[n_rows:], initial=n_rows, total=len(dataset.image_ids)):

        original_image = dataset.load_image(image_id)

        results = model.detect([original_image], verbose=0)

        r = results[0]
        n_detections = r['masks'].shape[2]

        detections = []
        source_image = dataset.image_info[image_id]['id']
        image_size = dataset.image_info[image_id]['size']

        for detection_index in range(n_detections):

            mask = r['masks'][:, :, detection_index]
            score = r['scores'][detection_index]
            class_id = r['class_ids'][detection_index]
            bbox = r['rois'][detection_index, :]

            label = dataset.get_source_class_id(class_id, 'openimages')


            #true_mask = oim.unmold_mask(mask, bbox, image_size)
            true_mask = mask

            rle_mask = oim.encode_binary_mask(true_mask)

            detections.extend([label, str(score), rle_mask.decode('ascii')])

        detections_str = " ".join(detections)
        row = ",".join([source_image, str(image_size[0]), str(image_size[1]), detections_str])
        
        fid.write(row+'\n')
        n_rows += 1


100%|██████████| 99999/99999 [25:25:26<00:00,  1.09it/s]   


In [ ]:
# complete with empty detections if necessary

In [ ]:
if n_rows < len(dataset.image_ids):
    with open(submission_file, 'a') as fid:
        for image_info in dataset.image_info[n_rows:]:
            fid.write(",".join([image_info['id'], '-1', '-1', '\n']))